## Conectarse a MySQL mediante Python

In [7]:
import mysql.connector
from mysql.connector import Error
import csv

Lo que primero se necesita es un diccionario para la configuracion de la conexion.

In [8]:
config = {
    'user': 'root',          # Usuario de MySQL
    'password': 'mascotas12@',  # Contraseña de MySQL
    'host': '127.0.0.1',     # Host (localhost)
    'database': 'mascotas',  # Nombre de la base de datos
    'raise_on_warnings': True
}

Ahora ejecutaremos el script para insertar datos

In [13]:
with open('data/new_data.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    print("Columnas detectadas:", reader.fieldnames)  # Imprime las columnas del CSV

Columnas detectadas: ['\ufeffDate', 'Geo Location', 'Name test', 'Value test', 'Pet sex', 'Pet age', 'Country', 'City', 'Postal code']


In [ ]:
try:
    # Conectar a la base de datos
    conexion = mysql.connector.connect(**config)

    if conexion.is_connected():

        cursor = conexion.cursor()
        with open('data/new_data.csv', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            
            for row in reader:
                # Insertar en codigo_postal
                sql_cod_postal = """
                INSERT INTO codigo_postal (cod_postal, cantidad_casos, latitud, longitud)
                VALUES (%s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE cantidad_casos = cantidad_casos + 1
                """
                cursor.execute(sql_cod_postal, (row["Postal code"], 1, 0, 0))
                
                # Insertar en mascota
                sql_mascota = """
                INSERT INTO mascota (ciudad, edad, sexo)
                VALUES (%s, %s, %s)
                """
                age = row["Pet age"]
                if age is not int():
                    age = 0
                cursor.execute(sql_mascota, (row["City"], age, row["Pet sex"]))
                id_mascota = cursor.lastrowid
                
                # Insertar en test
                sql_test = """
                INSERT INTO test (cod_postal, fecha, enfermedad, id_mascota, resultado)
                VALUES (%s, %s, %s, %s, %s)
                """
                cursor.execute(sql_test, (row["Postal code"], row["\ufeffDate"], row["Name test"], id_mascota, row["Value test"]))
                
        conexion.commit()
        print("Datos insertados correctamente desde el CSV.")

except Error as e:
    print(f"Error al conectarse a MySQL: {e}")

finally:
    # Cerrar la conexión
    if conexion.is_connected():
        cursor.close()
        conexion.close()
        print("Conexión cerrada")

Conexión cerrada


KeyError: 'Date'